In [1]:
import time
import asyncio
import pymongo
import json
from pymongo import MongoClient
from indy import anoncreds, did, ledger, pool, wallet, IndyError
import import_ipynb
import steward_wrapper
from datetime import datetime
from datetime import timedelta
from steward_wrapper import steward_registration, create_wallet, trustAnchor_registration, getting_verinym, send_nym, supplier_cred_schema, send_schema, supplier_cred_definition, get_schema, send_cred_def, device1_registration, supplier_create_credOffer, get_cred_def, retailerCreateProofRequest, supplierCreateCredential, deviceCreateCredRequest, get_credential_for_referent, prover_get_entities_from_ledger, verifier_get_entities_from_ledger
from policy_check_wrapper import device_policy_check

async def run():
    print("Getting started -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == 306:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    client = MongoClient(host='steward_db', port=27017, username='root', password='pass', authSource="admin")
    db = client["steward_db"]

    print("==============================")
    print("=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==")
    print("------------------------------")

    steward = {
        'name': "Sovrin Steward",
        'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
        'wallet_credentials': json.dumps({'key': 'steward_wallet_key'}),
        'pool': pool_['handle'],
        'seed': '000000000000000000000000Steward1'
    }
   
    await steward_registration(steward)
    
    
    print("==============================")
    print("== Getting Trust Anchor credentials - Supplier getting Verinym  ==")
    print("------------------------------")

    supplier = {
        'name': 'Supplier',
        'wallet_config': json.dumps({'id': 'supplier_wallet'}),
        'wallet_credentials': json.dumps({'key': 'supplier_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }
    
    x1 = time.time()
    await trustAnchor_registration(steward, supplier)
    print("In main code")
    x2 = time.time()
    print("Timestamp: Supplier Registration", x2-x1)   #supplier registration time

    print("Supplier: After registration", supplier)   # supplier wallet after registration

    print("==============================")
    print("== Getting Trust Anchor credentials - Retailer getting Verinym  ==")
    print("------------------------------")

    retailer = {
        'name': 'Retailer',
        'wallet_config': json.dumps({'id': 'retailer_wallet'}),
        'wallet_credentials': json.dumps({'key': 'retailer_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }

    await trustAnchor_registration(steward, retailer)

    print("==============================")
    print("=== Supplier Credential Schemas Setup ==")
    print("------------------------------")

    print("\"Supplier\" -> Create \"Supplier Attestation\" Schema")
    x1 = time.time()

   
    supplier_certificate = {
        'name': 'Supplier-Certificate',
        'version': '1.2',
        'attributes': ['first_name', 'last_name', 'degree', 'status','year', 'average', 'ssn'] # 'middle_name' ,'degree2','degree3','degree4' ,'status2','status3', 'status4'
    }

    await supplier_cred_schema(supplier,supplier_certificate)
    
    x2 = time.time()
    print("Timestamp: Supplier Credential Schema setup", x2-x1)

    print("Supplier: After credentail schema", supplier)

    print("==============================")
    print("=== Supplier Credential Definition Setup ==")
    print("------------------------------")

    x1 = time.time()
   
    supplier_certificate_cred_def = {
        'tag': 'TAG1',
        'type': 'CL',
        'config': {"support_revocation": False}
    }

    await supplier_cred_definition(supplier, supplier_certificate_cred_def)
   
    x2 = time.time()
    print("Timestamp: After Supplier Credential Def", x2-x1)

    print("Supplier: After credentail def", supplier)

    device1 = {
        'name': 'Device1',
        'wallet_config': json.dumps({'id': 'device_wallet'}),
        'wallet_credentials': json.dumps({'key': 'device_wallet_key'}),
        'pool': pool_['handle'],
        'manufacturer': 'Intel',
        'expiry': datetime.now() + timedelta(days=10),
        'temperature': 30,
        'location': 'London'
    }

    await device1_registration(device1)
    print("IoT Device: After registration", device1)   # supplier wallet after registration

    x1 = time.time()
    
    print("\"Supplier\" -> Create \"Supplier Certificate\" Credential Offer for Device")
    await supplier_create_credOffer(supplier)

    print("\"Supplier\" -> Send \"Supplier Certificate\" Credential Offer to Device1")
    device1['supplier_certificate_cred_offer'] = supplier['supplier_certificate_cred_offer']
    supplier_certificate_cred_offer_object = json.loads(device1['supplier_certificate_cred_offer'])

    device1['supplier_certificate_schema_id'] = supplier_certificate_cred_offer_object['schema_id']
    device1['supplier_certificate_cred_def_id'] = supplier_certificate_cred_offer_object['cred_def_id']

    print("\"Device1\" -> Create and store \"Device\" Master Secret in Wallet")
    device1['master_secret_id'] = await anoncreds.prover_create_master_secret(device1['wallet'], None)

    print("\"Device1\" -> Get \"Supplier Certificate\" Credential Definition from Ledger")
    (device1['supplier_cred_def_id'], device1['supplier_cred_def']) = \
        await get_cred_def(device1['pool'], device1['did'], device1['supplier_certificate_cred_def_id'])

    
    print("\"Device1\" -> Create \"Supplier Certificate\" Credential Request for Supplier")
    await deviceCreateCredRequest(device1)

    print("\"Device1\" -> Send \"Supplier Certificate\" Credential Request to Supplier")
    supplier['supplier_certificate_cred_request'] = device1['supplier_certificate_cred_request']

    device1['supplier_certificate_cred_values'] = json.dumps({
        "first_name": {"raw": "Alice", "encoded": "1139481716457488690172217916278103335"},
        "last_name": {"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
        "degree": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
        "status": {"raw": "graduated", "encoded": "2213454313412354"},
        "ssn": {"raw": "123-45-6789", "encoded": "3124141231422543541"},
        "year": {"raw": "2015", "encoded": "2015"},
        "average": {"raw": "5", "encoded": "5"},
        # "middle_name":{"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
        # "degree2": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
        # "degree3": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
        # "degree4": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
        # "status2": {"raw": "graduated", "encoded": "2213454313412354"},
        # "status3": {"raw": "graduated", "encoded": "2213454313412354"},
        # "status4": {"raw": "graduated", "encoded": "2213454313412354"},
    })

    supplier['device_supplier_certificate_cred_values'] = device1['supplier_certificate_cred_values']

    print("\"Supplier\" -> Create \"Supplier Certificate\" Credential for Device1")
    await supplierCreateCredential(supplier)
    x2 = time.time()

    print("Timestamp: After VC Creation", x2-x1)
    print("Supplier: After credentail Issue", supplier)
    


    print("\"Supplier\" -> Send \"Supplier Certificate\" Credential to Device1")
    
    device1['supplier_certificate_cred'] = supplier['supplier_certificate_cred']

    print("\"Device1\" -> Store \"Transcript\" Credential from Supplier")
    _, device1['supplier_certificate_cred_def'] = await get_cred_def(device1['pool'], device1['did'],
                                                         device1['supplier_certificate_cred_def_id'])

    await anoncreds.prover_store_credential(device1['wallet'], None, device1['supplier_certificate_cred_request_metadata'],
                                            device1['supplier_certificate_cred'], device1['supplier_certificate_cred_def'], None)
    
    print("IoTDevice: After credentail Issue", device1)

    print("\n=====================================================================")
    print("== Prove Certificate to Retailer - Supplier Crdential proving ==")

    print("\"Retailer\" -> Create \"Access\" Proof Request")
    await retailerCreateProofRequest(retailer, supplier)

    print("\"Retailer\" -> Send \"Verification\" Proof Request to Alice")
    device1['verification_proof_request'] = retailer['verification_proof_request']

    search_for_verification_proof_request = \
        await anoncreds.prover_search_credentials_for_proof_req(device1['wallet'],
                                                                device1['verification_proof_request'], None)

    cred_for_attr1 = await get_credential_for_referent(search_for_verification_proof_request, 'attr1_referent')
    cred_for_attr2 = await get_credential_for_referent(search_for_verification_proof_request, 'attr2_referent')
    cred_for_attr3 = await get_credential_for_referent(search_for_verification_proof_request, 'attr3_referent')
    cred_for_attr4 = await get_credential_for_referent(search_for_verification_proof_request, 'attr4_referent')
    cred_for_attr5 = await get_credential_for_referent(search_for_verification_proof_request, 'attr5_referent')
    # cred_for_attr7 = await get_credential_for_referent(search_for_verification_proof_request, 'attr7_referent')
    # cred_for_attr8 = await get_credential_for_referent(search_for_verification_proof_request, 'attr8_referent')
    # cred_for_attr9 = await get_credential_for_referent(search_for_verification_proof_request, 'attr9_referent')
    # cred_for_attr10 = await get_credential_for_referent(search_for_verification_proof_request, 'attr10_referent')
    # cred_for_attr11 = await get_credential_for_referent(search_for_verification_proof_request, 'attr11_referent')
    # cred_for_attr12 = await get_credential_for_referent(search_for_verification_proof_request, 'attr12_referent')
    # cred_for_attr13 = await get_credential_for_referent(search_for_verification_proof_request, 'attr13_referent')
    cred_for_predicate1 = \
        await get_credential_for_referent(search_for_verification_proof_request, 'predicate1_referent')

    await anoncreds.prover_close_credentials_search_for_proof_req(search_for_verification_proof_request)

    device1['creds_for_verification_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                                cred_for_attr2['referent']: cred_for_attr2,
                                                cred_for_attr3['referent']: cred_for_attr3,
                                                cred_for_attr4['referent']: cred_for_attr4,
                                                cred_for_attr5['referent']: cred_for_attr5,
                                                # cred_for_attr7['referent']: cred_for_attr7,
                                                # cred_for_attr8['referent']: cred_for_attr8,
                                                # cred_for_attr9['referent']: cred_for_attr9,
                                                # cred_for_attr10['referent']: cred_for_attr10,
                                                # cred_for_attr11['referent']: cred_for_attr11,
                                                # cred_for_attr12['referent']: cred_for_attr12,
                                                # cred_for_attr13['referent']: cred_for_attr13,
                                                cred_for_predicate1['referent']: cred_for_predicate1
                                                }
    x1 = time.time()
    device1['schemas'], device1['cred_defs'], device1['revoc_states'] = \
        await prover_get_entities_from_ledger(device1['pool'], device1['did'],
                                              device1['creds_for_verification_proof'], device1['name'])
    
    print("\"Device1\" -> Create \"Job-Application\" Proof")
    device1['verification_requested_creds'] = json.dumps({
        'self_attested_attributes': {
            'attr1_referent': 'Alice',
            'attr2_referent': 'Garcia',
            'attr6_referent': '123-45-6789',
            #'attr10_referent': 'Garcia',
        },
        'requested_attributes': {
            'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True},
            'attr4_referent': {'cred_id': cred_for_attr4['referent'], 'revealed': True},
            'attr5_referent': {'cred_id': cred_for_attr5['referent'], 'revealed': True},
            # 'attr7_referent': {'cred_id': cred_for_attr7['referent'], 'revealed': True},
            # 'attr8_referent': {'cred_id': cred_for_attr8['referent'], 'revealed': True},
            # 'attr9_referent': {'cred_id': cred_for_attr9['referent'], 'revealed': True},
            # 'attr11_referent': {'cred_id': cred_for_attr11['referent'], 'revealed': True},
            # 'attr12_referent': {'cred_id': cred_for_attr12['referent'], 'revealed': True},
            # 'attr13_referent': {'cred_id': cred_for_attr13['referent'], 'revealed': True}
        },
        'requested_predicates': {'predicate1_referent': {'cred_id': cred_for_predicate1['referent']}}
    })

    device1['verification_proof'] = \
        await anoncreds.prover_create_proof(device1['wallet'], device1['verification_proof_request'],
                                            device1['verification_requested_creds'], device1['master_secret_id'],
                                            device1['schemas'], device1['cred_defs'], device1['revoc_states'])

    print("\"Device1\" -> Send \"Verification\" Proof to Retailer")
    retailer['verification_proof'] = device1['verification_proof']
    verification_proof_object = json.loads(retailer['verification_proof'])

    retailer['schemas_for_verification'], retailer['cred_defs_for_verification'], \
    retailer['revoc_ref_defs_for_verification'], retailer['revoc_regs_for_verification'] = \
        await verifier_get_entities_from_ledger(retailer['pool'], retailer['did'],
                                                verification_proof_object['identifiers'], retailer['name'])
 
    print("\"Retailer\" -> Verify \"Verification\" Proof from Device1")
    assert 'Bachelor of Science, Marketing' == \
           verification_proof_object['requested_proof']['revealed_attrs']['attr3_referent']['raw']
    assert 'graduated' == \
           verification_proof_object['requested_proof']['revealed_attrs']['attr4_referent']['raw']
    assert '123-45-6789' == \
           verification_proof_object['requested_proof']['revealed_attrs']['attr5_referent']['raw']
    # assert 'Bachelor of Science, Marketing' == \
    #        verification_proof_object['requested_proof']['revealed_attrs']['attr7_referent']['raw']
    # assert 'Bachelor of Science, Marketing' == \
    #        verification_proof_object['requested_proof']['revealed_attrs']['attr8_referent']['raw']
    # assert 'Bachelor of Science, Marketing' == \
    #        verification_proof_object['requested_proof']['revealed_attrs']['attr9_referent']['raw']
    # assert 'graduated' == \
    #        verification_proof_object['requested_proof']['revealed_attrs']['attr11_referent']['raw']
    # assert 'graduated' == \
    #        verification_proof_object['requested_proof']['revealed_attrs']['attr12_referent']['raw']
    # assert 'graduated' == \
    #        verification_proof_object['requested_proof']['revealed_attrs']['attr13_referent']['raw']

    assert 'Alice' == verification_proof_object['requested_proof']['self_attested_attrs']['attr1_referent']
    assert 'Garcia' == verification_proof_object['requested_proof']['self_attested_attrs']['attr2_referent']
    assert '123-45-6789' == verification_proof_object['requested_proof']['self_attested_attrs']['attr6_referent']
    #assert 'Garcia' == verification_proof_object['requested_proof']['self_attested_attrs']['attr10_referent']

    assert await anoncreds.verifier_verify_proof(retailer['verification_proof_request'], retailer['verification_proof'],
                                                 retailer['schemas_for_verification'],
                                                 retailer['cred_defs_for_verification'],
                                                 retailer['revoc_ref_defs_for_verification'],
                                                 retailer['revoc_regs_for_verification'])

    x2 = time.time()
    print("Timestamp: After VC Verification", x2-x1)

    device1['authentication_expiry'] = datetime.now() + timedelta(days=1)
    device1['access_count'] = 0
    device1['authenticated'] = True
    device1['authorised'] = False

    policies = {
      'valid_manufacturers': ['Intel', 'AMD'],
      'valid_locations': ['London'],
      'max_temperature': 40,
      'max_access_count': 1
    }

    for i in range(10):
      x1 = time.time()
      #print("Timestamp: Before Policy Check", x1)
      policy_check_result = await device_policy_check(device1, policies)
      policy_check_result = await device_policy_check(device1, policies)
      policy_check_result = await device_policy_check(device1, policies)
      policy_check_result = await device_policy_check(device1, policies)
      policy_check_result = await device_policy_check(device1, policies)
      device1['authenticated'] = policy_check_result['authenticated']
      device1['authorised'] = policy_check_result['authorised']
      x2 = time.time()
      print("Timestamp: After Policy Check", x2-x1)
    

    print("=======================================================")

    print("\"Sovrin Steward\" -> Close and Delete wallet")
    await wallet.close_wallet(steward['wallet'])
    await wallet.delete_wallet(steward['wallet_config'], steward['wallet_credentials'])

    print("\"Supplier\" -> Close and Delete wallet")
    await wallet.close_wallet(supplier['wallet'])
    await wallet.delete_wallet(supplier['wallet_config'], supplier['wallet_credentials'])

    print("\"Device1\" -> Close and Delete wallet")
    await wallet.close_wallet(device1['wallet'])
    await wallet.delete_wallet(device1['wallet_config'], device1['wallet_credentials'])

    print("Close and Delete pool")
    await pool.close_pool_ledger(pool_['handle'])
    await pool.delete_pool_ledger_config(pool_['name'])
    print("=======================================================")
    print("Getting started done")

if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    await run()
    time.sleep(1)  # FIXME waiting for libindy thread complete


importing Jupyter notebook from steward_wrapper.ipynb
importing Jupyter notebook from policy_check_wrapper.ipynb
Getting started -> started
Open Pool Ledger: pool1
=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==
------------------------------
"Sovrin Steward" -> Create wallet
"Sovrin Steward" -> Create and store in Wallet DID from seed
== Getting Trust Anchor credentials - Supplier getting Verinym  ==
------------------------------
"Supplier" -> Create wallet
In getting verinym
In getting verinym1
In getting verinym2
In getting verinym3
In main code
Timestamp: Supplier Registration 3.1133456230163574
Supplier: After registration {'name': 'Supplier', 'wallet_config': '{"id": "supplier_wallet"}', 'wallet_credentials': '{"key": "supplier_wallet_key"}', 'pool': 2, 'role': 'TRUST_ANCHOR', 'wallet': 7, 'did': '3NMtb3cuSMf3484nuymw67', 'key': '2HsE5gZcXoACkBKHVQyKpPhVi2niFQ7tTF4muT3xEzYW'}
== Getting Trust Anchor credentials - Retailer getting Verinym  ==
-----

IoT Device: After registration {'name': 'Device1', 'wallet_config': '{"id": "device_wallet"}', 'wallet_credentials': '{"key": "device_wallet_key"}', 'pool': 2, 'manufacturer': 'Intel', 'expiry': datetime.datetime(2022, 6, 19, 14, 35, 37, 293043), 'temperature': 30, 'location': 'London', 'wallet': 20, 'did': 'EDPoZ48gDHbzwEsgQZeLJk', 'key': '8CiCW459qQrF4TMh7jSyzVuHmBsG43p7jP36YhXWweZj'}
"Supplier" -> Create "Supplier Certificate" Credential Offer for Device
"Supplier" -> Send "Supplier Certificate" Credential Offer to Device1
"Device1" -> Create and store "Device" Master Secret in Wallet
"Device1" -> Get "Supplier Certificate" Credential Definition from Ledger
"Device1" -> Create "Supplier Certificate" Credential Request for Supplier
"Device1" -> Send "Supplier Certificate" Credential Request to Supplier
"Supplier" -> Create "Supplier Certificate" Credential for Device1
Timestamp: After VC Creation 0.24759626388549805
Supplier: After credentail Issue {'name': 'Supplier', 'wallet_config

"Device1" -> Send "Verification" Proof to Retailer
"Retailer" -> Get Schema from Ledger
"Retailer" -> Get Credential Definition from Ledger
"Retailer" -> Verify "Verification" Proof from Device1
Timestamp: After VC Verification 0.428480863571167
Timestamp: After Policy Check 1.3828277587890625e-05
Timestamp: After Policy Check 7.3909759521484375e-06
Timestamp: After Policy Check 6.9141387939453125e-06
Timestamp: After Policy Check 6.9141387939453125e-06
Timestamp: After Policy Check 6.9141387939453125e-06
Timestamp: After Policy Check 6.9141387939453125e-06
Timestamp: After Policy Check 7.152557373046875e-06
Timestamp: After Policy Check 6.9141387939453125e-06
Timestamp: After Policy Check 6.9141387939453125e-06
Timestamp: After Policy Check 7.152557373046875e-06
"Sovrin Steward" -> Close and Delete wallet
"Supplier" -> Close and Delete wallet
"Device1" -> Close and Delete wallet
Close and Delete pool
Getting started done
